# 線上 PDF 壓縮工具 (Online PDF Compressor)

這個 Colab Notebook 檔案利用 ghostscript 這個強大的工具來進行 PDF 壓縮，並提供了一個簡單易用的介面，讓您可以輕鬆上傳多個 PDF 檔案，進行壓縮，然後將壓縮後的檔案打包下載。


In [ ]:
# @title 歡迎使用線上 PDF 壓縮工具
# @markdown 這個 Colab 筆記本可以幫助您壓縮多個 PDF 檔案，以減少檔案大小。
# @markdown ---
# @markdown **使用步驟：**
# @markdown 1.  執行第一個儲存格**「步驟一：安裝 Ghostscript」**，以安裝必要的工具。
# @markdown 2.  執行第二個儲存格**「步驟二：上傳您的 PDF 檔案」**，並選擇您想要壓縮的所有 PDF 檔案。
# @markdown 3.  在第三個儲存格**「步驟三：選擇壓縮品質並執行壓縮」**中，選擇您想要的壓縮等級，然後執行它。
# @markdown 4.  執行最後一個儲存格**「步驟四：下載壓縮後的 PDF 檔案」**，所有壓縮過的檔案將會被打包成一個 ZIP 檔供您下載。
# @markdown ---
# @markdown **注意：**
# @markdown *   此服務完全免費。
# @markdown *   您的檔案將在 Colab 的暫存空間中處理，並在執行階段結束後自動刪除，確保您的隱私。
# @markdown *   壓縮效果會因原始 PDF 的內容而異，包含大量圖片的 PDF 檔案通常會有最顯著的壓縮效果。

In [ ]:
# @title 步驟一：安裝 Ghostscript
# @markdown 請執行此儲存格以安裝 PDF 壓縮所需的 `ghostscript` 工具。
# @markdown 這個過程可能需要幾秒鐘。

!apt-get update
!apt-get install -y ghostscript
print("Ghostscript 安裝完成！")

In [ ]:
# @title 步驟二：上傳您的 PDF 檔案
# @markdown 執行此儲存格後，請點擊「選擇檔案」按鈕，然後選取一個或多個您想要壓縮的 PDF 檔案。
# @markdown 上傳完成後，檔案列表會顯示在下方。

import os
from google.colab import files

# 創建用於存放上傳檔案和壓縮後檔案的資料夾
upload_folder = 'uploaded_pdfs'
compressed_folder = 'compressed_pdfs'

if not os.path.exists(upload_folder):
    os.makedirs(upload_folder)
if not os.path.exists(compressed_folder):
    os.makedirs(compressed_folder)

# 清空舊檔案
for f in os.listdir(upload_folder):
    os.remove(os.path.join(upload_folder, f))
for f in os.listdir(compressed_folder):
    os.remove(os.path.join(compressed_folder, f))


uploaded = files.upload()

# 將上傳的檔案移動到指定資料夾
for filename, content in uploaded.items():
    with open(os.path.join(upload_folder, filename), 'wb') as f:
        f.write(content)
    print(f'已上傳檔案： {filename}')

if not os.listdir(upload_folder):
    print("您沒有上傳任何檔案。")
else:
    print("\n所有檔案上傳成功！")

In [ ]:
# @title 步驟三：選擇壓縮品質並執行壓縮
# @markdown 請從下拉選單中選擇一個壓縮品質等級，然後執行此儲存格。 [4, 14, 18]
# @markdown - **低品質 (screen):** 檔案最小，適合螢幕閱讀，不適合印刷。
# @markdown - **中等品質 (ebook):** 檔案大小與品質的良好平衡，適合電子書。
# @markdown - **高品質 (printer):** 檔案較大，適合印刷。
# @markdown - **最高品質 (prepress):** 檔案最大，保留最多細節，適合專業印刷。

import subprocess
import shutil

quality_level = 'prepress'  # @param ["screen", "ebook", "printer", "prepress"]

input_folder = 'uploaded_pdfs'
output_folder = 'compressed_pdfs'
pdf_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.pdf')]

if not pdf_files:
    print("找不到任何已上傳的 PDF 檔案。請先執行步驟二。")
else:
    for filename in pdf_files:
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        original_size = os.path.getsize(input_path) / 1024  # KB

        command = [
            'gs',
            '-sDEVICE=pdfwrite',
            '-dCompatibilityLevel=1.4',
            f'-dPDFSETTINGS=/{quality_level}',
            '-dNOPAUSE',
            '-dQUIET',
            '-dBATCH',
            f'-sOutputFile={output_path}',
            input_path
        ]

        try:
            subprocess.run(command, check=True)
            compressed_size = os.path.getsize(output_path) / 1024  # KB
            reduction = 100 - (compressed_size / original_size * 100)
            print(f"成功壓縮 '{filename}'")
            print(f"  原始大小: {original_size:.2f} KB")
            print(f"  壓縮後大小: {compressed_size:.2f} KB")
            print(f"  大小減少: {reduction:.2f}%")
        except subprocess.CalledProcessError as e:
            print(f"壓縮 '{filename}' 時發生錯誤: {e}")
            # 如果壓縮失敗，將原始檔案複製過去
            shutil.copy(input_path, output_path)
            print(f"  已將原始檔案複製到輸出資料夾。")
        print("-" * 20)
    print("\n所有檔案處理完成！")

In [ ]:
# @title 步驟四：下載壓縮後的 PDF 檔案
# @markdown 執行此儲存格會將所有壓縮過的 PDF 檔案打包成一個 ZIP 檔，並自動開始下載。
# @markdown 如果您只有一個檔案，它也會被打包成 ZIP 檔。

import shutil

output_folder = 'compressed_pdfs'

if not os.listdir(output_folder):
    print("沒有可供下載的壓縮檔案。請先執行步驟三。")
else:
    zip_filename = 'compressed_pdfs.zip'
    shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', output_folder)

    print(f"已將所有壓縮檔案打包成 '{zip_filename}'。")
    print("準備下載...")
    files.download(zip_filename)